In [9]:
# INITIALIZE
import pandas as pd
#!/usr/bin/env python3
import os
from dotenv import load_dotenv
import typing
import fmpsdk
from helper import to_percentage


# Actual API key is stored in a .env file.  Not good to store API key directly in script.
load_dotenv()
apikey = os.environ.get("apikey")






In [10]:
# this block will be pasted at constant.py at the end of the development
from enum import Enum
class INCOMESTATEMENT(Enum):
    CALENDAR_YEAR='calendarYear'
    GROSSPROFIT_RATIO='grossProfitRatio'
    NETINCOME_RATIO='netIncomeRatio'
    GROSS_PROFIT='grossProfit'
    RD='researchAndDevelopmentExpenses'
    SGA_EXPENSE='sellingGeneralAndAdministrativeExpenses'
    DEPRECIATION='depreciationAndAmortization'
    OPERATING_INCOME='operatingIncome'
    INTEREST_EXPENSE='interestExpense'


class BALANCESHEET(Enum):
    CALENDAR_YEAR='calendarYear'
    LONGTERM_DEBT='longTermDebt'
    TOTAL_LIABILITIES='totalLiabilities'
    TOTAL_STOCKHOLDEREQUITIES='totalStockholdersEquity'

class BALANCEGROWTH(Enum):
    CALENDAR_YEAR='calendarYear'
    RETAINEDEARNINGS_GROWTH='growthRetainedEarnings'


class CASHFLOWSTATEMENT(Enum):
    CALENDAR_YEAR='calendarYear'
    CAPEX='capitalExpenditure'
    COMMONSTOCK_REPURCHASE='commonStockRepurchased'
    DIVIDEND_PAID='dividendsPaid'

class PERIOD(Enum):
    QUARTER='quarter'
    ANNUAL='annual'

In [11]:
# Get data from SDK
symbol: str = input("Please enter stock symbol ")
period:str=input("Enter 'quarter' and 'annual'")
period=period.strip()
# financial statement
income_statement=fmpsdk.income_statement(apikey=apikey,period=period,symbol=symbol)
balance_sheet=fmpsdk.balance_sheet_statement(apikey=apikey,period=period,symbol=symbol)
cashflow_statement=fmpsdk.cash_flow_statement(apikey=apikey,period=period,symbol=symbol)

# growth data
if period==PERIOD.ANNUAL.value:
    balance_growth=fmpsdk.balance_sheet_statement_growth(apikey=apikey,symbol=symbol)


In [12]:
# CASHFLOW STATEMENT
cashflow_columns=[
    CASHFLOWSTATEMENT.CALENDAR_YEAR.value,
    CASHFLOWSTATEMENT.CAPEX.value,
    CASHFLOWSTATEMENT.COMMONSTOCK_REPURCHASE.value,
    CASHFLOWSTATEMENT.DIVIDEND_PAID.value
]

if period==PERIOD.QUARTER.value:
    cashflow_columns.append('period')

cashflow_frame=pd.DataFrame(data=cashflow_statement,columns=cashflow_columns)

In [13]:
# BALANCE SHEET
balance_columns=[BALANCESHEET.CALENDAR_YEAR.value,
                BALANCESHEET.LONGTERM_DEBT.value,
                BALANCESHEET.TOTAL_LIABILITIES.value,
                BALANCESHEET.TOTAL_STOCKHOLDEREQUITIES.value]

if period==PERIOD.QUARTER.value:
    balance_columns.append('period')
balance_frame=pd.DataFrame(data=balance_sheet,columns=balance_columns)

# ratio
balance_frame['liability_shareholder ratio']=balance_frame[BALANCESHEET.TOTAL_LIABILITIES.value]/balance_frame[BALANCESHEET.TOTAL_STOCKHOLDEREQUITIES.value]


# displaying options
balance_columns_todrop=[
    BALANCESHEET.TOTAL_LIABILITIES.value,
    BALANCESHEET.TOTAL_STOCKHOLDEREQUITIES.value
]

balance_frame=balance_frame.drop(labels=balance_columns_todrop,axis='columns')


In [14]:
# BALANCE GROWTH
# sheet analysis only happend once a year
if period==PERIOD.ANNUAL.value:
    balance_growth_columns=[BALANCEGROWTH.CALENDAR_YEAR.value,BALANCEGROWTH.RETAINEDEARNINGS_GROWTH.value]
    balance_growth_frame=pd.DataFrame(data=balance_growth,columns=balance_growth_columns)
    balance_growth_frame[BALANCEGROWTH.RETAINEDEARNINGS_GROWTH.value]=balance_growth_frame[BALANCEGROWTH.RETAINEDEARNINGS_GROWTH.value].map(to_percentage)

In [15]:
# INCOME STATEMENT
# from constant import INCOMESTATEMENT


# selecting fields from original data structure and import them into pandas.DataFrame
income_columns=[INCOMESTATEMENT.CALENDAR_YEAR.value,
                INCOMESTATEMENT.GROSSPROFIT_RATIO.value,
                INCOMESTATEMENT.NETINCOME_RATIO.value,
                INCOMESTATEMENT.GROSS_PROFIT.value,
                INCOMESTATEMENT.RD.value,
                INCOMESTATEMENT.SGA_EXPENSE.value,
                INCOMESTATEMENT.DEPRECIATION.value,
                INCOMESTATEMENT.INTEREST_EXPENSE.value,
                INCOMESTATEMENT.OPERATING_INCOME.value]
if period==PERIOD.QUARTER.value:
    income_columns.append('period')
income_frame=pd.DataFrame(data=income_statement,columns=income_columns)

# extract necessary ratio from income statement
income_frame["RD_Ratio"]=(income_frame[INCOMESTATEMENT.RD.value]/income_frame[INCOMESTATEMENT.GROSS_PROFIT.value]).map(to_percentage)
income_frame["SGA_Ratio"]=(income_frame[INCOMESTATEMENT.SGA_EXPENSE.value]/income_frame[INCOMESTATEMENT.GROSS_PROFIT.value]).map(to_percentage)
income_frame["Depreciation_Ratio"]=(income_frame[INCOMESTATEMENT.DEPRECIATION.value]/income_frame[INCOMESTATEMENT.GROSS_PROFIT.value]).map(to_percentage)
income_frame[INCOMESTATEMENT.GROSSPROFIT_RATIO.value]=income_frame[INCOMESTATEMENT.GROSSPROFIT_RATIO.value].map(to_percentage)
income_frame[INCOMESTATEMENT.NETINCOME_RATIO.value]=income_frame[INCOMESTATEMENT.NETINCOME_RATIO.value].map(to_percentage)
income_frame["interestExpense_operatingIncome_Ratio"]=(income_frame[INCOMESTATEMENT.INTEREST_EXPENSE.value]/income_frame[INCOMESTATEMENT.OPERATING_INCOME.value]).map(to_percentage)

# drop unnecessary columns for display
income_columns_todrop=[INCOMESTATEMENT.RD.value,
                       INCOMESTATEMENT.SGA_EXPENSE.value,
                       INCOMESTATEMENT.DEPRECIATION.value,
                       INCOMESTATEMENT.INTEREST_EXPENSE.value,
                       INCOMESTATEMENT.OPERATING_INCOME.value
                       ]
income_frame=income_frame.drop(labels=income_columns_todrop,axis='columns')

In [16]:
# MAIN TABLE
joinkeys=['calendarYear','period'] if period==PERIOD.QUARTER.value else ['calendarYear']
table=pd.merge(left=income_frame,right=balance_frame,on=joinkeys)
table=pd.merge(left=table,right=cashflow_frame,on=joinkeys)
if period==PERIOD.ANNUAL.value:
    table=pd.merge(left=table,right=balance_growth_frame,on=joinkeys)
#ratio
table['debt_grossProfit ratio']=(table[BALANCESHEET.LONGTERM_DEBT.value]/table[INCOMESTATEMENT.GROSS_PROFIT.value]).round(2)
table['capex_grossProfit_rario']=(table[CASHFLOWSTATEMENT.CAPEX.value].abs()/table[INCOMESTATEMENT.GROSS_PROFIT.value]).map(to_percentage)

# drop unimportant column
table_columns_todrop=[
    INCOMESTATEMENT.GROSS_PROFIT.value,
    BALANCESHEET.LONGTERM_DEBT.value,
    CASHFLOWSTATEMENT.CAPEX.value
]
table=table.drop(labels=table_columns_todrop,axis='columns')
table.style.set_caption(f'Table 1: stats for {symbol}')




,calendarYear,grossProfitRatio,netIncomeRatio,RD_Ratio,SGA_Ratio,Depreciation_Ratio,interestExpense_operatingIncome_Ratio,liability_shareholder ratio,commonStockRepurchased,dividendsPaid,growthRetainedEarnings,debt_grossProfit ratio,capex_grossProfit_rario
0,2022,59.56 %,44.90 %,12.11 %,4.71 %,34.75 %,1.05 %,0.680423,-871566000,-285234185000,38.32 %,0.640000,80.81 %
1,2021,51.63 %,37.10 %,15.22 %,5.43 %,53.81 %,0.83 %,0.717050,0,-265786399000,20.03 %,0.770000,103.65 %
2,2020,53.10 %,38.67 %,15.40 %,5.00 %,48.57 %,0.37 %,0.492031,0,-259303805000,19.15 %,0.390000,73.33 %
3,2019,46.05 %,32.27 %,18.55 %,5.70 %,62.36 %,0.87 %,0.396389,0,-259303805000,-3.15 %,0.080000,95.34 %
4,2018,48.27 %,34.05 %,17.25 %,5.27 %,62.59 %,0.80 %,0.246080,0,-207443044000,11.62 %,0.110000,64.81 %
5,2017,50.62 %,35.10 %,16.32 %,5.49 %,55.39 %,0.86 %,0.308203,0,-181512663000,15.05 %,0.190000,67.88 %
6,2016,50.09 %,35.26 %,15.00 %,5.41 %,47.13 %,0.85 %,0.357319,0,-155582283000,19.87 %,0.320000,70.15 %
7,2015,48.65 %,36.34 %,15.97 %,5.59 %,54.23 %,0.99 %,0.355974,0,-116683481000,26.94 %,0.470000,63.79 %
8,2014,49.52 %,34.59 %,15.04 %,6.36 %,53.00 %,1.09 %,0.429878,0,-77785851000,35.96 %,0.570000,77.41 %
9,2013,47.06 %,31.52 %,17.07 %,8.32 %,57.91 %,1.26 %,0.490001,0,-77773307000,26.20 %,0.750000,103.35 %
